In [ ]:
from pyhive import hive
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
try:
    con = hive.Connection(host="localhost", port=10000)

    cur = con.cursor()
    cur.execute("create Database {}".format("CpuLogData"))
    print("Database Created Successfully")

except Exception as e:
    print(e)

Table Creation

In [ ]:
try:
    con = hive.Connection(host="localhost", port=10000, database="CpuLogData")

    cur = con.cursor()
    cur.execute("create table data(Date_Time  DATE ,Cpu_Count int,Cpu_Working_Time string,Cpu_idle_Time string,cpu_percent string,Usage_cpu_count string,number_of_software_interrupts_since_boot string,number_of_system_calls_since_boot string,number_of_interrupts_since_boot string,cpu_avg_load_over_1_min string,cpu_avg_load_over_5_min string,cpu_avg_load_over_15_min string,system_total_memory string,system_used_memory string,system_free_memory string,system_active_memory string,system_inactive_memory string,system_buffers_memory string,system_cached_memory string,system_shared_memory string,system_avalible_memory string,disk_total_memory string,disk_used_memory string,disk_free_memory string,disk_read_count string,disk_write_count string,disk_read_bytes string,disk_write_bytes string,time_spent_reading_from_disk string,time_spent_writing_to_disk string,time_spent_doing_actual_Input_Output string,number_of_bytes_sent string,number_of_bytes_received string,number_of_packets_sent string,number_of_packets_recived string,Total_number_of_errors_while_receiving string,total_number_of_errors_while_sending string,total_number_of_incoming_packets_which_were_dropped string,total_number_of_outgoing_packets_which_were_dropped string,boot_time string,user_name string,keyboard int,mouse int,technology string,files_changed int) row format delimited fields terminated by ',' stored as textfile location 'hdfs://localhost:9000/NewOne' tblproperties('skip.header.line.count'='1')")
    print("Table Created Successfully")

except Exception as e:
    print(e)

To get the User Count who are Working

In [ ]:
try:
    con = hive.Connection(host="localhost", port=10000, database="CpuLogData")
    user_working_count = pd.read_sql("select user_name ,count('') as total from data where keyboard !=0 or mouse !=0 group by user_name", con)
    print(user_working_count)

except Exception as e:
    print(e)

Plotting Bar Graph for above data

In [ ]:
try:
    user_working_count.plot.bar(x='user_name', y='total' ,color='r')
    plt.ylim(0,600)
    plt.xlabel("UserName")
    plt.ylable("Working Count")
    plt.show()
except Exception as e:
    print(e)    


Getting Average Working Seconds of Each User

In [ ]:
avg_work_sec=pd.read_sql("select user_name,((((count(''))*5)*60)/6) as average_working_sec from data where keyboard!=0 or mouse!=0 group by user_name", con)
print(avg_work_sec)

Converting Average Work Seconds to Average Hour and getting Highest, Lowest Average Working Hour.

In [ ]:
average_work_hour = avg_work_sec[["user_name","average_working_sec"]]
average_work_hour["average_working_sec"]= pd.to_datetime(average_work_hour['average_working_sec'] ,unit='s').dt.strftime("%H:%M")
average_work_hour.rename(columns={'average_working_sec':'average_working_hour'}, inplace=True)
print(average_work_hour,"\n")

print("Highest Average Working Hour")
highest_average_work_hour=average_work_hour.max()
print(highest_average_work_hour, "\n")

print("Lowest Average Working Hour")
lowest_average_work_hour=average_work_hour.min()
print(lowest_average_work_hour)


BarPlot for above data.

Getting Counts For User Sitting Idle

In [ ]:
try:
    con = hive.Connection(host="localhost", port=10000, database="cpulog")
    user_idle_count = pd.read_sql("select user_name ,count('') as total from data where keyboard =0 or mouse !=0 group by user_name", con)
    print(user_idle_count)

except Exception as e:
    print(e)

Getting Average Idle Seconds of All Users.

In [ ]:
avg_idle_sec=pd.read_sql("select user_name,((((count(''))*5)*60)/6) as average_idle_sec from data where keyboard=0 or mouse=0 group by user_name", con)
print(avg_idle_sec)

Converting Average Idle Seconds to Average idle Hours and finding the Highest, Lowest Idle Hrs.

In [ ]:
average_idle_hour = avg_idle_sec[["user_name","average_idle_sec"]]
average_idle_hour["average_idle_sec"]= pd.to_datetime(average_idle_hour['average_idle_sec'] ,unit='s').dt.strftime("%H:%M")
average_idle_hour.rename(columns={'average_idle_sec':'average_idle_hour'}, inplace=True)
print(average_idle_hour,"\n")

print("Highest Average Idle Hour")
highest_average_idle_hour=average_idle_hour.max()
print(highest_average_idle_hour, "\n")

print("Lowest Average Working Hour")
lowest_average_idle_hour=average_idle_hour.min()
print(lowest_average_idle_hour)

Bar Plot for Above Data

In [ ]:
total = user_working_count['total']
plt.figure(figsize = (10,10))
sns.barplot(x= 'average_idle_hour',y=total,data = average_idle_hour,hue='user_name')
plt.title("Avg Idle hours by user")
plt.show()